# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: The PEFT technique used was Lora
* Model: The model used was gpt-2 as suggested
* Evaluation approach: The evaluation approach was accuracy
* Fine-tuning dataset: The Dataset chosen for fine-tuning was Emotion:
https://huggingface.co/datasets/dair-ai/emotion

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [ ]:
!pip install -q peft
!pip install -q transformers
!pip install -q "datasets==2.15.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
!python --version

Python 3.10.12


In [ ]:
from peft import LoraConfig, PeftConfig, get_peft_model, AutoPeftModelForCausalLM, TaskType, PeftModel, AutoPeftModelForSequenceClassification
from transformers import AutoTokenizer, GPT2ForSequenceClassification, AutoModelForCausalLM, AutoModelForSequenceClassification, DataCollatorWithPadding, Trainer, TrainingArguments
from datasets import load_dataset
import torch
import numpy as np
import pandas as pd

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# Load and Split DataSet

In [ ]:
# load emotion dataset:
# https://huggingface.co/datasets/dair-ai/emotion

splits = ["train", "test", "validation"]

train = load_dataset("dair-ai/emotion", split="train")
test = load_dataset("dair-ai/emotion", split="test")
valid = load_dataset("dair-ai/emotion", split="validation")

data={}
data["train"]=train.shuffle(seed=42).select(range(800)) #8000
data["test"]=test.shuffle(seed=42).select(range(100)) #1000
data["validation"]=valid.shuffle(seed=42).select(range(100)) #1000

# View the dataset characteristics
print("train size:",data["train"])
print("test size:",data["test"])
print("valid size:",data["validation"])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

train size: Dataset({
    features: ['text', 'label'],
    num_rows: 800
})
test size: Dataset({
    features: ['text', 'label'],
    num_rows: 100
})
valid size: Dataset({
    features: ['text', 'label'],
    num_rows: 100
})


# Sample train dataset

In [ ]:
data["train"].shuffle(seed=42).select(range(10))[:10]

{'text': ['ive slowed down i take time to listen to my child and be in the moment and not feel like i need to immediately update my status on fb about the cute thing she did',
  'i had then these were truly terrifying and still feel shaken and uneasy because of them',
  'i am feeling a bit adventurous i put on red lipstick rouge artist and intense from make up for ever',
  'i struggled with feeling like myself because myself liked bands and the s and david hockney and photography and collecting things and no body really understood those things because no body does understand you when you re',
  'id feel completely lost without him',
  'i really feel rotten and my ear hurts so bad but i still managed to work out days and really push the intensity',
  'i feel nervous about trying something new during a lesson or if my horse shies at something',
  'i feel like im gonna be so greedy with him cuz i just love him so much',
  'i think people are merely lacking of professionalism and ethics wh

# Create a Tokenizer and Pre-process Datasets

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
#tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
MAX_LEN=100

def preprocess_function(examples):
    """Preprocess the emotion dataset by returning tokenized examples."""
    return tokenizer(examples["text"], padding="max_length", truncation=True, return_tensors="pt", max_length=MAX_LEN)


tokenized_data = {}
for split in splits:
    tokenized_data[split] = data[split].map(preprocess_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
#nputs = tokenizer(train["text"][0], return_tensors="pt")
#utputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
#print(tokenizer.batch_decode(outputs))

# Show the first example of the tokenized training set
print(tokenized_data["train"][0]["input_ids"])

[4514, 16259, 287, 262, 1499, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256]


## Load and set up the model

We will now load the model and freeze most of the parameters of the model: everything except the classification head.

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained( #use chatpt2
    "gpt2",
    num_labels=6,
    ignore_mismatched_sizes=True, #
    id2label={0: "sadness", 1: "joy", 2:"love", 3:"anger",4:"fear", 5:"surprise"},  # For converting predictions to strings
    label2id={"sadness": 0, "joy": 1, "love": 2, "anger": 3, "fear": 4, "surprise": 5},
)
#model = AutoModelForCausalLM.from_pretrained("gpt2") #use chatpt2
model.config.pad_token_id = tokenizer.pad_token_id
# Freeze all the parameters of the base model
# Hint: Check the documentation at https://huggingface.co/transformers/v4.2.2/training.html
for param in model.base_model.parameters():
    param.requires_grad = False

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(model)

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=6, bias=False)
)


In [ ]:
#tokenizer.pad_token = tokenizer.eos_token

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}


# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./data/sentiment_analysis",
        learning_rate=2e-3,
        # Reduce the batch size if you don't have enough memory
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=1,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer, padding="max_length"),
    compute_metrics=compute_metrics,
)
#trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


# Evaluates model performance

In [ ]:
test_df = pd.DataFrame(tokenized_data["test"])
test_df = test_df[["text", "label"]]
test_pred = trainer.predict(tokenized_data["test"])
test_df["predicted_label"] = np.argmax(test_pred[0], axis=1)

In [ ]:
test_acc =(test_df["label"] == test_df["predicted_label"]).mean()
print('Test Acuracy: ',test_acc)

Test Acuracy:  0.05


In [ ]:
trainer.evaluate()

{'eval_loss': 6.344542026519775,
 'eval_accuracy': 0.06,
 'eval_runtime': 7.9978,
 'eval_samples_per_second': 12.503,
 'eval_steps_per_second': 3.126}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [ ]:
config = LoraConfig( inference_mode=False, r=18, lora_alpha=32, lora_dropout=0.1,modules_to_save=["classifier"], bias='lora_only',use_rslora=True) #Criando uma configuração PEFT
#config = LoraConfig()

lora_model = get_peft_model(model, config) #Criando um modelo PEFT

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:1059: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [ ]:
lora_model.print_trainable_parameters()

trainable params: 294,912 || all params: 124,739,328 || trainable%: 0.23642263007862283


# Train the model for 15 epochs

In [ ]:
trainer = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir="./data/sentiment_analysis",
        learning_rate=2e-3,
        # Reduce the batch size if you don't have enough memory
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        #label_names = ["start_positions", "end_positions"],
        num_train_epochs=15,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        label_names=["labels"], #
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer, padding="max_length"),
    compute_metrics=compute_metrics,
)
#trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.222510,0.590000
2,No log,0.739919,0.770000
3,1.393900,0.845042,0.680000
4,1.393900,0.797331,0.750000
5,0.858800,0.945720,0.810000
6,0.858800,0.627398,0.780000
7,0.858800,0.765344,0.790000
8,0.416000,0.948047,0.800000
9,0.416000,0.795930,0.830000
10,0.277700,0.776924,0.840000


TrainOutput(global_step=3000, training_loss=0.5273500531514486, metrics={'train_runtime': 2649.8297, 'train_samples_per_second': 4.529, 'train_steps_per_second': 1.132, 'total_flos': 6293091778560000.0, 'train_loss': 0.5273500531514486, 'epoch': 15.0})

# Save config model

In [ ]:
config=lora_model.config
config

GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "id2label": {
    "0": "sadness",
    "1": "joy",
    "2": "love",
    "3": "anger",
    "4": "fear",
    "5": "surprise"
  },
  "initializer_range": 0.02,
  "label2id": {
    "anger": 3,
    "fear": 4,
    "joy": 1,
    "love": 2,
    "sadness": 0,
    "surprise": 5
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 50256,
  "problem_type": "single_label_classification",
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_

# Evaluates model performance

In [ ]:
val_df = pd.DataFrame(tokenized_data["validation"])
val_df = val_df[["text", "label"]]
val_pred = trainer.predict(tokenized_data["validation"])
val_df["predicted_label"] = np.argmax(val_pred[0], axis=1)

In [ ]:
val_acc =(val_df["label"] == val_df["predicted_label"]).mean()
print('validation Acuracy: ',val_acc)

validation Acuracy:  0.78


In [ ]:
test_pred = trainer.predict(tokenized_data["test"])
test_df["predicted_label"] = np.argmax(test_pred[0], axis=1)

In [ ]:
test_acc =(test_df["label"] == test_df["predicted_label"]).mean()
print('Test Acuracy: ',test_acc)

Test Acuracy:  0.9


In [ ]:
trainer.evaluate()

{'eval_loss': 0.6273983120918274,
 'eval_accuracy': 0.78,
 'eval_runtime': 9.5016,
 'eval_samples_per_second': 10.525,
 'eval_steps_per_second': 2.631,
 'epoch': 15.0}

# Save model weights

In [ ]:
#lora_model = lora_model.merge_and_unload()
#trainer.save_model("model_lora")
lora_model.save_pretrained("gpt-lora")
tokenizer.save_pretrained("lora-tokenizer")

('lora-tokenizer/tokenizer_config.json',
 'lora-tokenizer/special_tokens_map.json',
 'lora-tokenizer/vocab.json',
 'lora-tokenizer/merges.txt',
 'lora-tokenizer/added_tokens.json',
 'lora-tokenizer/tokenizer.json')

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

# Load model weights after training

In [ ]:
#adapter_model_name = "gpt-lora"
model_name="gpt-lora" #"model_lora"
#lora_model = AutoPeftModelForCausalLM.from_pretrained(model_name).to(device)
#lora_model2 = AutoPeftModelForSequenceClassification.from_pretrained(model_name).to(device)
lora_model2 = GPT2ForSequenceClassification.from_pretrained(model_name, config=config)
#lora_model2 = PeftModel.from_pretrained(model2, adapter_model_name)

#merged_model2 =lora_model2.merge_and_unload()
tokenizer = AutoTokenizer.from_pretrained("lora-tokenizer")

OUT:Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight'] You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.

In [ ]:
#lora_model2.resize_token_embeddings(len(tokenizer))
#tokenizer.pad_token = tokenizer.eos_token
#lora_model2.config.pad_token_id = lora_model2.config.eos_token_id

In [ ]:
trainer = Trainer(
    model=lora_model2,
    args=TrainingArguments(
        output_dir="./data/sentiment_analysis",
        learning_rate=2e-3,
        # Reduce the batch size if you don't have enough memory
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        #label_names = ["start_positions", "end_positions"],
        num_train_epochs=10,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        label_names=["labels"], #
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer, padding="max_length"),
    compute_metrics=compute_metrics,
)
#trainer.train()

# Evaluates model performance after fine tuning

In [ ]:
trainer.evaluate()

{'eval_loss': 3.2905619144439697,
 'eval_accuracy': 0.17,
 'eval_runtime': 9.3529,
 'eval_samples_per_second': 10.692,
 'eval_steps_per_second': 2.673}

In [ ]:
#tokenizer.pad_token = tokenizer.eos_token
test_pred = trainer.predict(tokenized_data["test"])
test_df["predicted_label"] = np.argmax(test_pred[0], axis=1)
test_acc =(test_df["label"] == test_df["predicted_label"]).mean()
print('Test Acuracy: ',test_acc)

Test Acuracy:  0.11
